# Compare model outputs of AICORE DL Model


### Imports 

In [52]:
import plotly.express as px
import pandas as pd
import os
import glob
from pathlib import Path
import yaml
import datetime
import numpy as np
import seaborn as sns

# workaround buggy autocomplete
%config Completer.use_jedi = False

In [32]:
def calc_rank(df, feature='basename'):
    basenames = pd.unique(df[feature])
    for basename in basenames:
        idx = (df[df.basename == basename]).sort_values(by='IoU', ascending=False).index
        df.loc[idx, 'rank'] = np.arange(1, 101, dtype=np.int)
    return df

In [28]:
def print_overview(df):
    sites = pd.unique(df.region)
    for site in sites:
        data = df[df.basename.str.contains(site)]
        print('\n', site)
        print(data.groupby(by='basename').max().sort_values(by='a-b', ascending=False).set_index('a-b')['Epoch'])

### Import Data

In [ ]:
df_val = pd.read_pickle('df_val_filtered.pkl')

In [ ]:
df_train = pd.read_pickle('df_train_filtered.pkl')

### Check data completeness 

In [33]:
print_overview(df_val)


 Banks_Island
a-b
UnetPlusPlus_resnet50     100
UnetPlusPlus_resnet34     100
UnetPlusPlus_resnet101    100
UNet_resnet50             100
UNet_resnet34             100
UNet_resnet101            100
DeepLabV3_resnet50        100
DeepLabV3_resnet34        100
DeepLabV3_resnet101       100
Name: Epoch, dtype: int64

 Herschel
a-b
UnetPlusPlus_resnet50     100
UnetPlusPlus_resnet34     100
UnetPlusPlus_resnet101    100
UNet_resnet50             100
UNet_resnet34             100
UNet_resnet101            100
DeepLabV3_resnet50        100
DeepLabV3_resnet34        100
DeepLabV3_resnet101       100
Name: Epoch, dtype: int64

 Horton
a-b
UnetPlusPlus_resnet50     100
UnetPlusPlus_resnet34     100
UnetPlusPlus_resnet101    100
UNet_resnet50             100
UNet_resnet34             100
UNet_resnet101            100
DeepLabV3_resnet50        100
DeepLabV3_resnet34        100
DeepLabV3_resnet101       100
Name: Epoch, dtype: int64

 Kolguev
a-b
UnetPlusPlus_resnet50     100
UnetPlusPlus_resnet34

### Data + Feature Engineering 

In [34]:
df_val = calc_rank(df_val)
df_train = calc_rank(df_train)

### Data Analysis 

#### Best runs result per run
1. Maximum
2. 95 %
3. 50 %

#### Show best individual run per config sorted by IoU:

#### Plot Best (1, 5, 10, 20, 50) IoU for each configuraion 

In [137]:
query = 'rank in (1, 5, 10, 20, 50)'
cm = sns.light_palette("green", as_cmap=True)
df_vals_by_basename = df_val.pivot_table(values='IoU', index='rank', columns='basename').query(query).T
df_vals_by_basename.style.background_gradient(cmap=cm)

rank,1.0,5.0,10.0,20.0,50.0
basename,,,,,
CV6__V__Banks_Island__DeepLabV3_resnet101_FocalLoss_sh6_2021-04-11_06-46-03,0.181262,0.145836,0.039931,0.012721,0.003159
CV6__V__Banks_Island__DeepLabV3_resnet34_FocalLoss_sh6_2021-04-03_09-50-15,0.198617,0.126145,0.094006,0.062944,0.029504
CV6__V__Banks_Island__DeepLabV3_resnet50_FocalLoss_sh6_2021-04-05_08-42-51,0.306233,0.085294,0.032669,0.016210,0.004237
CV6__V__Banks_Island__UNet_resnet101_FocalLoss_sh6_2021-03-29_13-13-38,0.306444,0.166405,0.144575,0.059466,0.010129
CV6__V__Banks_Island__UNet_resnet34_FocalLoss_sh6_2021-03-24_00-43-54,0.276679,0.193598,0.120957,0.017486,0.000327
CV6__V__Banks_Island__UNet_resnet50_FocalLoss_sh6_2021-03-24_19-12-38,0.091947,0.061816,0.021933,0.009645,0.001980
CV6__V__Banks_Island__UnetPlusPlus_resnet101_FocalLoss_sh6_2021-03-31_09-41-01,0.259004,0.110111,0.044700,0.015352,0.001853
CV6__V__Banks_Island__UnetPlusPlus_resnet34_FocalLoss_sh6_2021-04-26_16-24-13,0.317857,0.232007,0.112624,0.035645,0.005871
CV6__V__Banks_Island__UnetPlusPlus_resnet50_FocalLoss_sh6_2021-04-28_02-01-27,0.350788,0.126413,0.079953,0.039203,0.013755


#### Show results of best config per region 

In [138]:
site = 'Horton'
query1 = 'rank in (1, 5, 10, 20, 50)'
stack = []
for site in sites:
    query2 = f'region == "{site}"'
    filtered = df_val.query(query2).query(query).pivot(index='basename', columns='rank', values='IoU')
    out = filtered.sort_values(by=1.0, axis=0, ascending=False).iloc[0]
    stack.append(out)
region_best = pd.concat(stack, axis=1).T

cm = sns.light_palette("green", as_cmap=True)
region_best.style.background_gradient(cmap=cm)

rank,1.0,5.0,10.0,20.0,50.0
CV6__V__Banks_Island__UnetPlusPlus_resnet50_FocalLoss_sh6_2021-04-28_02-01-27,0.350788,0.126413,0.079953,0.039203,0.013755
CV6__V__Herschel__DeepLabV3_resnet34_FocalLoss_sh6_2021-04-03_16-34-51,0.388486,0.328877,0.315564,0.300387,0.272052
CV6__V__Horton__UnetPlusPlus_resnet101_FocalLoss_sh6_2021-04-01_05-19-44,0.546923,0.535739,0.513225,0.497303,0.434148
CV6__V__Kolguev__UnetPlusPlus_resnet101_FocalLoss_sh6_2021-04-01_15-08-25,0.475656,0.446787,0.429816,0.396055,0.334650
CV6__V__Lena__UnetPlusPlus_resnet34_FocalLoss_sh6_2021-04-27_14-28-50,0.577838,0.514543,0.495771,0.476423,0.397816
CV6__V__Tuktoyuktuk__UnetPlusPlus_resnet50_FocalLoss_sh6_2021-03-28_09-17-42,0.145230,0.088152,0.081210,0.072353,0.050628


#### show stats by architecture/backbone 

In [148]:
query = 'rank in (1, 5, 10, 20, 50)'
cm = sns.light_palette("green", as_cmap=True)
df_vals_by_basename = df_val.pivot_table(values='IoU', index='rank', columns='a-b', aggfunc='mean').T
df_vals_by_basename.style.background_gradient(cmap=cm)

rank,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,59.0,60.0,61.0,62.0,63.0,64.0,65.0,66.0,67.0,68.0,69.0,70.0,71.0,72.0,73.0,74.0,75.0,76.0,77.0,78.0,79.0,80.0,81.0,82.0,83.0,84.0,85.0,86.0,87.0,88.0,89.0,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,100.0
a-b,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DeepLabV3_resnet101,0.289305,0.276403,0.272526,0.269960,0.262268,0.245669,0.239657,0.237136,0.235360,0.233627,0.232113,0.230772,0.226576,0.223566,0.220113,0.218903,0.217833,0.217561,0.216767,0.215559,0.213048,0.212232,0.210580,0.209883,0.208646,0.207217,0.206129,0.204218,0.202488,0.201721,0.201356,0.199943,0.199611,0.199314,0.198624,0.196713,0.195383,0.194703,0.194012,0.192741,0.191318,0.190725,0.189537,0.188067,0.187015,0.186128,0.185533,0.184600,0.184236,0.183717,0.182675,0.181473,0.181263,0.179778,0.179040,0.178603,0.177792,0.177019,0.176242,0.175755,0.174743,0.172964,0.170771,0.169466,0.166704,0.165960,0.164857,0.163611,0.162581,0.162034,0.161391,0.160505,0.159309,0.157653,0.155866,0.155074,0.153922,0.152059,0.150452,0.143603,0.141329,0.140272,0.136879,0.132285,0.129249,0.125678,0.123832,0.120390,0.114823,0.110124,0.106080,0.100170,0.085760,0.080536,0.079398,0.070995,0.053291,0.039828,0.019400,0.001089
DeepLabV3_resnet34,0.327551,0.304439,0.295580,0.288478,0.283062,0.279360,0.275731,0.271092,0.268742,0.265365,0.263718,0.260996,0.258251,0.256870,0.255286,0.254429,0.252742,0.250451,0.249872,0.247548,0.246461,0.244989,0.243826,0.242244,0.241533,0.240198,0.238546,0.237717,0.236754,0.235375,0.234243,0.232959,0.232181,0.230160,0.229793,0.227929,0.226970,0.226025,0.224726,0.223392,0.221837,0.221114,0.220218,0.219096,0.218153,0.217584,0.216665,0.214659,0.213678,0.210394,0.209699,0.206676,0.205620,0.203999,0.202780,0.202282,0.200517,0.198586,0.197424,0.196573,0.195191,0.194560,0.193030,0.192012,0.190832,0.189883,0.187725,0.185953,0.183847,0.182066,0.181410,0.177680,0.176242,0.173715,0.172589,0.171830,0.169542,0.164218,0.161362,0.159957,0.156015,0.153084,0.149979,0.148796,0.146893,0.142515,0.141522,0.138873,0.136409,0.128936,0.121664,0.115956,0.113482,0.103754,0.090176,0.073027,0.049471,0.040880,0.027741,0.008705
DeepLabV3_resnet50,0.344229,0.317382,0.291715,0.287325,0.283274,0.275743,0.271890,0.264904,0.261566,0.258024,0.255116,0.252611,0.250897,0.248493,0.247085,0.245370,0.244535,0.243635,0.242686,0.240517,0.239069,0.238062,0.237187,0.234842,0.233537,0.232477,0.231472,0.230141,0.229565,0.228686,0.226965,0.226335,0.225195,0.224530,0.223092,0.221304,0.219826,0.218708,0.217877,0.217485,0.216115,0.215060,0.211169,0.210260,0.208372,0.207243,0.206608,0.204675,0.202931,0.201133,0.199302,0.198411,0.196470,0.195649,0.193022,0.191547,0.186814,0.184880,0.182025,0.180238,0.177582,0.173518,0.171569,0.170174,0.168687,0.167860,0.167179,0.165876,0.162856,0.161738,0.161070,0.159030,0.157586,0.156305,0.153603,0.152238,0.149524,0.147622,0.145066,0.141873,0.139865,0.137309,0.132080,0.130583,0.128201,0.124539,0.120221,0.115865,0.113766,0.109386,0.099941,0.094139,0.086077,0.080920,0.075569,0.064543,0.054234,0.045840,0.043793,0.024644
UNet_resnet101,0.353172,0.320213,0.308411,0.302442,0.294261,0.291982,0.285973,0.282742,0.278766,0.276287,0.273844,0.266525,0.263897,0.262526,0.255270,0.253304,0.252427,0.248425,0.242712,0.239524,0.236905,0.234852,0.234084,0.232752,0.230465,0.228803,0.228275,0.227033,0.225076,0.224383,0.222820,0.220336,0.219205,0.217618,0.216683,0.214400,0.212582,0.210526,0.209227,0.207696,0.205868,0.204692,0.203476,0.202290,0.201015,0.199323,0.197513,0.196888,0.196252,0.195074,0.194437,0.193924,0.192129,0.190721,0.189175,0.188155,0.187442,0.187230,0.186261,0.184625,0.183951,0.181617,0.180536,0.179453,0.178753,0.177192,0.176245,0.174281,0.172099,0.171

## Plots 

In [149]:
# put data into plottable shape
id_vars = ['Epoch', 'basename', 'type', 'architecture', 'a-b', 'backbone']

df_train_melt = df_train.melt(id_vars=id_vars, value_vars=['IoU']).sort_values(by='Epoch')
df_val_melt = df_val.melt(id_vars=id_vars, value_vars=['IoU']).sort_values(by='Epoch')
# merge
df_melt = pd.concat([df_val_melt])
#df_melt = pd.concat([df_train_melt, df_val_melt])

### Training data 

In [ ]:
#px.line(df_melt, x='Epoch' , y='value' , color='basename', line_group='type')
df_train_melt = df_train.melt(id_vars=id_vars, value_vars=['IoU']).sort_values(by='Epoch')
p = px.line(df_train_melt, x='Epoch' , y='value' , color='basename', line_group='architecture')
p.update_layout(legend_orientation='h', )

### Validation data 

#### Performance per Epoch IoU

In [ ]:
df_val_melt = df_val.melt(id_vars=id_vars, value_vars=['IoU']).sort_values(by='Epoch')
p = px.line(df_val_melt.sort_values(by=['Epoch', 'architecture', 'backbone']), x='Epoch' , y='value' , color='basename', line_dash='architecture', line_group='backbone')
p.update_layout(legend_orientation='h', )

In [ ]:
df_val_melt_PR = df_val.melt(id_vars=id_vars, value_vars=['P-R']).sort_values(by='Epoch')
p = px.line(df_val_melt_PR.sort_values(by=['Epoch', 'architecture', 'backbone']), x='Epoch' , y='value' , color='basename', line_dash='architecture', line_group='backbone')
p.update_layout(legend_orientation='h', )

#### Plot mean values of configurations 

In [ ]:
group_vals = ['Epoch', 'architecture']
df_train_melt['ds'] = df_train_melt.apply(lambda x: x.architecture, axis=1)
mean_vals = df_train_melt.groupby(group_vals).mean().reset_index(drop=False)
px.line(mean_vals, x='Epoch', y='value', color=group_vals[1])

### Backbone 

#### Mean performance 

In [ ]:
mean_vals

In [ ]:
group_vals = ['Epoch', 'backbone']
df_val_melt['ds'] = df_val_melt.apply(lambda x: x.architecture, axis=1)
mean_vals = df_val_melt.groupby(group_vals).mean().reset_index(drop=False)
px.line(mean_vals, x='Epoch', y='value', color=group_vals[1])

In [ ]:
group_vals = ['Epoch', 'architecture']
df_val_melt['ds'] = df_val_melt.apply(lambda x: x.architecture, axis=1)
mean_vals = df_val_melt.groupby(group_vals).mean().reset_index(drop=False)
px.line(mean_vals, x='Epoch', y='value', color=group_vals[1])

### Indivual configs per site 

#### Regional comparison UNet 

In [ ]:
backbone = 'resnet50'
architecture = 'UNet'
stack_height = 6
loss = 'FocalLoss'
data_sources = '[planet, ndvi, tcvis, relative_elevation, slope]'

In [ ]:
query = f'backbone == "{backbone}" & architecture == "{architecture}" & stack_height == "{stack_height}" & loss_function == "{loss}"'
plot_data = df_val.query(query)
px.line(plot_data, x='Epoch', y='IoU', color='basename')

#### Regional comparison DeepLabV3

In [ ]:
backbone = 'resnet50'
architecture = 'DeepLabV3'
stack_height = 6
loss = 'FocalLoss'
data_sources = '[planet, ndvi, tcvis, relative_elevation, slope]'

In [ ]:
query = f'backbone == "{backbone}" & architecture == "{architecture}" & stack_height == "{stack_height}" & loss_function == "{loss}"'
plot_data = df_val.query(query)
px.line(plot_data, x='Epoch', y='IoU', color='basename')

#### Regional comparison UnetPlusPlus

In [ ]:
backbone = 'resnet50'
architecture = 'UnetPlusPlus'
stack_height = 6
loss = 'FocalLoss'
data_sources = '[planet, ndvi, tcvis, relative_elevation, slope]'

In [ ]:
query = f'backbone == "{backbone}" & architecture == "{architecture}" & stack_height == "{stack_height}" & loss_function == "{loss}"'
plot_data = df_val.query(query)
px.line(plot_data, x='Epoch', y='IoU', color='basename')

#### Sorted IoU by config

In [ ]:

pd.unique(plot_data.basename)

In [ ]:
#for region in regions:
query = f'region == "Horton"'
plot_data = df_val.query(query)
basenames = pd.unique(plot_data.basename)
for basename in basenames:
    idx = (plot_data[plot_data.basename == basename]).sort_values(by='IoU', ascending=False).index
    plot_data.loc[idx, 'rank'] = np.arange(100, dtype=np.int)
metric = 'IoU'
px.line(plot_data.sort_values(by=metric), x='rank', y=metric, color='basename')

In [ ]:
regions

In [ ]:
query = f'region == "Lena"'
plot_data = df_val.query(query)
for basename in basenames:
    idx = (plot_data[plot_data.basename == basename]).sort_values(by='IoU', ascending=False).index
    plot_data.loc[idx, 'rank'] = np.arange(100, dtype=np.int)
metric = 'IoU'
px.line(plot_data.sort_values(by=metric), x='rank', y=metric, color='basename')

In [ ]:
site = 'V__Horton'
#df_val[df_val.basename.str.contains(site)].sort_values(by='IoU', ascending=False).drop_duplicates().head(5)

In [ ]:
plot_data = df_val[df_val.basename.str.contains(site)]
px.line(plot_data, x='Epoch', y='IoU', color='basename')
#data.groupby(by='basename').max().sort_values(by='IoU', ascending=False).set_index('a-b')